In [1]:
import time
# import tqdm
from tqdm.notebook import tqdm  # Import the notebook version of tqdm

from datasets import load_dataset
import pandas as pd
import numpy as np
import huggingface_hub
from huggingface_hub import HfFileSystem
hffs = HfFileSystem()
from concurrent.futures import ThreadPoolExecutor, as_completed

import transformers
transformers.logging.set_verbosity_error()
from transformers import AutoTokenizer


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
df = pd.read_parquet("./original-data-00000-of-00099.parquet")

In [3]:
df.head()

,text,id,dump,url,file_path,language,language_score,token_count,score,int_score
0,"The Independent Jane\nFor all the love, romanc...",<urn:uuid:0d8a309d-25c5-405d-a08a-c11239f0d717>,CC-MAIN-2013-20,http://austenauthors.net/the-independent-jane,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.974320,845,2.750000,3
1,Taking Play Seriously\nBy ROBIN MARANTZ HENIG\...,<urn:uuid:316c7af5-14e1-4d0b-9576-753e17ef2cc5>,CC-MAIN-2013-20,http://query.nytimes.com/gst/fullpage.html?res...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.961459,1055,2.562500,3
2,How do you get HIV?\nHIV can be passed on when...,<urn:uuid:a3e140cd-7f25-48c9-a2f0-a7d0b1954e0d>,CC-MAIN-2013-20,http://www.childline.org.uk/Explore/SexRelatio...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.966757,136,3.125000,3
3,CTComms sends on average 2 million emails mont...,<urn:uuid:c337bcd8-6aa1-4f2d-8c48-b916442ebbee>,CC-MAIN-2013-20,http://www.ctt.org/resource_centre/getting_sta...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.910602,3479,3.234375,3
4,Hold the salt: UCLA engineers develop revoluti...,<urn:uuid:c0b175bb-65fb-420e-a881-a80b91d00ecd>,CC-MAIN-2013-20,http://www.environment.ucla.edu/water/news/art...,s3://commoncrawl/crawl-data/CC-MAIN-2013-20/se...,en,0.924981,1115,2.812500,3


In [4]:
# reading in the fetched file
cdf = pd.read_parquet("./embedded-data-00000-of-00099.parquet")


In [5]:
cdf.head()

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,id,url,score,dump,embedding
168141,2,[SEP],[102],1,<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>,https://www.wired.com/2005/10/mapping-katrinas...,3.531250,CC-MAIN-2020-05,"[0.048728548, 0.06860847, -0.22434144, -0.0079..."
22349,2,[SEP],[102],1,<urn:uuid:974d6c7a-e045-4fff-ab94-d56b3ad96a0d>,https://preferhome.com/blog/elder-care-for-par...,2.671875,CC-MAIN-2017-26,"[0.048728548, 0.06860847, -0.22434144, -0.0079..."
172425,4,[SEP],[102],1,<urn:uuid:23827efd-395d-49fb-bf12-157f7c867ae2>,http://artemaya.com/artist_life_bp.html,3.437500,CC-MAIN-2020-05,"[0.04872725, 0.06852399, -0.22433548, -0.00797..."
104574,2,[SEP],[102],1,<urn:uuid:760669a8-a183-46d1-b23c-3cf1c86074e4>,https://byjus.com/chemistry/daltons-atomic-the...,4.281250,CC-MAIN-2020-05,"[0.04872725, 0.06852399, -0.22433548, -0.00797..."
49273,2,[SEP],[102],1,<urn:uuid:56a693bc-9678-4760-8d10-91a46ae8b655>,http://www.youthinkgreen.org/en/unifying-globa...,2.640625,CC-MAIN-2020-05,"[0.04872725, 0.06852399, -0.22433548, -0.00797..."


In [6]:
df[df['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]

,text,id,dump,url,file_path,language,language_score,token_count,score,int_score
61706,"GULFPORT, Mississippi – As a backhoe scoops up...",<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>,CC-MAIN-2020-05,https://www.wired.com/2005/10/mapping-katrinas...,s3://commoncrawl/crawl-data/CC-MAIN-2020-05/se...,en,0.976334,909,3.53125,4


In [7]:
cdf[cdf['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]

,chunk_index,chunk_text,chunk_tokens,chunk_token_count,id,url,score,dump,embedding
168141,2,[SEP],[102],1,<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>,https://www.wired.com/2005/10/mapping-katrinas...,3.53125,CC-MAIN-2020-05,"[0.048728548, 0.06860847, -0.22434144, -0.0079..."
168140,1,". "" hurricane force winds extended 125 miles f...","[1012, 1000, 7064, 2486, 7266, 3668, 8732, 266...",451,<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>,https://www.wired.com/2005/10/mapping-katrinas...,3.53125,CC-MAIN-2020-05,"[0.054695774, 0.05058043, -0.22701378, -0.0780..."
168139,0,"[CLS] gulfport, mississippi – as a backhoe sco...","[101, 6084, 6442, 1010, 5900, 1516, 2004, 1037...",500,<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>,https://www.wired.com/2005/10/mapping-katrinas...,3.53125,CC-MAIN-2020-05,"[0.059140284, 0.03848576, -0.2385424, -0.07934..."


In [11]:
cdf[cdf['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]["chunk_text"]

168141                                                [SEP]
168140    . " hurricane force winds extended 125 miles f...
168139    [CLS] gulfport, mississippi – as a backhoe sco...
Name: chunk_text, dtype: object

In [12]:
# Get the full text string of the rows and display it
full_text = df[df['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]['text'].values[0]
print(full_text)


GULFPORT, Mississippi – As a backhoe scoops up a massive pile of torn-up drywall, furniture and soggy carpets in front of the home, Gary Bryan and two other surveyors examine the spot.
They're one of several crews hired by the government to painstakingly map flood levels from Louisiana to Florida. Across 90,000 square miles, crews are tracking down hundreds of markings posted by earlier teams right after the storm. They record their location and elevation.
Their goal is to create an atlas. Emergency planners want to learn not only what happened during Katrina, but to prepare for what might happen with future storms. How high might the water surge next time?
The house where the crew found one of the markers is a whole half-mile from the Gulf of Mexico. But it's near Bernard Bayou, which rose when the hurricane roared onto land Aug. 29.
"I'm from Florida. We had three eyes come pretty close to us and we didn't even get teased compared to this," Bryan said, comparing Katrina's wrath to th

In [15]:
cfull_text = cdf[cdf['id'] == "<urn:uuid:4c22d5ca-1ffb-4b58-a113-1bf47d6953ee>"]["chunk_text"].values[1]

In [16]:
print(cfull_text)

. " hurricane force winds extended 125 miles from katrina's center, compared to 60 miles for camille, he said. also, katrina's eye was 32 miles wide even though a storm of its intensity normally has an eye that is 10 miles wide. scientists know much more about surge today than they did in 1969 when camille hit. they can predict with reasonable accuracy what the surge will be. but the unpredictable nature of storms makes it difficult to say far in advance which areas will be flooded and how much higher the wind - whipped waves on top of the surge will be. for example, hurricane ivan last year was predicted to hit west of mobile, alabama. if it had, it would have sent 12 feet of storm surge into the area and 4 feet over the pensacola area. but just hours before landfall, it shifted and hit east of the city. it spared mobile, but sent a 10 - foot surge over the florida panhandle, said stephen baig, a storm surge expert at the national hurricane center in miami. in the case of katrina, est

In [17]:
tokenizer =  AutoTokenizer.from_pretrained("bert-base-uncased")

In [18]:
tokens = tokenizer.encode(full_text)

In [19]:
len(tokens)

901

In [20]:
# weird chunks are because of the 50 overlap. 901 tokens means one 500 chunk, then start at 450 and get 451 tokens. 
# but next start_index is 900 so we get one more chunk of 1 tokens